In [5]:
from typing import List, Optional
from pydantic import BaseModel

class Variant(BaseModel):
    name: str
    sku: str
    available: bool
    price: float

class Product(BaseModel):
    id: int
    title: str
    variants: Optional[List[Variant]] # Optional[] because not all products have variantsimport BaseModel

In [8]:
# raises type validation error: the 'variants' argument receives a list of strings, instead of list of Variants
item = Product(
    id=123,
    title='tshirt',
    variants=['should', 'not', 'work']
)

ValidationError: 3 validation errors for Product
variants -> 0
  value is not a valid dict (type=type_error.dict)
variants -> 1
  value is not a valid dict (type=type_error.dict)
variants -> 2
  value is not a valid dict (type=type_error.dict)

In [11]:
# does not raise an error, because we declared the 'variants' attribute to be *Optional*
item = Product(
    id=123,
    title='tshirt',
)

print(item.variants)

None


In [15]:
# does not raise an error, because we declared the 'variants' attribute to be *Optional*
item = Product(
    id=123,
    title='tshirt',
    variants=[Variant(name='blue_tshirt', sku='1384314', available=True, price=19.99)]
)
item

Product(id=123, title='tshirt', variants=[Variant(name='blue_tshirt', sku='1384314', available=True, price=19.99)])

Pydantic tries to cast data to the correct type, if possible.

Example:

```
str    -> int   
'123'  -> 123

str    -> bool
'true' -> True
```

In [25]:
var = Variant(name='blue_tshirt', sku=12345, available='t', price='12.99')
var

Variant(name='blue_tshirt', sku='12345', available=True, price=12.99)

Note that this could lead to unexpected behavior. Consider casting a string to boolean:

```
str    -> bool
----------------
'true' -> True
'0'    -> False
'1'    -> True
'f'    -> False
't'    -> True
```

## Validators

In [29]:
from typing import List, Optional
from pydantic import BaseModel, validator

class Variant(BaseModel):
    name: str
    sku: str
    available: bool
    price: float

    @validator('sku')
    def sku_length(cls, value): # cls = class method
        if len(value) != 7:
            raise ValueError('SKU must be 7 chars long.')
        return value

class Product(BaseModel):
    id: int
    title: str
    variants: Optional[List[Variant]] # Optional[] because not all products have variantsimport BaseModel

In [32]:
good_variant = Variant(
    name='blue_tshirt',
    sku='1234567',
    available=True,
    price=15.99
)
good_variant

Variant(name='blue_tshirt', sku='1234567', available=True, price=15.99)

In [33]:
bad_variant = Variant(
    name='blue_tshirt',
    sku='123456',
    available=True,
    price=15.99
)

ValidationError: 1 validation error for Variant
sku
  SKU must be 7 chars long. (type=value_error)